***Go Mind***

In [1]:
%pip install gymnasium torch tqdm matplotlib
%pip install pygame

  Using cached gymnasium-1.2.2-py3-none-any.whl.metadata (10 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached cloudpickle-3.1.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl.metadata (558 bytes)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached gymnasium-1.2.2-py3-none-any.whl (952 kB)
   ---------------------------------------- 0.0/110.9 MB ? eta -:--:--
    ---


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached pygame-2.6.1-cp312-cp312-win_amd64.whl.metadata (13 kB)
Using cached pygame-2.6.1-cp312-cp312-win_amd64.whl (10.6 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import gymnasium as gym
import matplotlib.pyplot as plt
from torch.distributions import Categorical
from tqdm import tqdm

In [ ]:
import pygame
import sys

BOARD_SIZE = 19
CELL_SIZE = 30
PADDING = 40
STONE_RADIUS = 12

EMPTY = 0
BLACK = 1
WHITE = 2

HAS_PLAYER = True
TRAIN = True

if not TRAIN:
    pygame.init()
    screen = pygame.display.set_mode(
        (BOARD_SIZE * CELL_SIZE + PADDING * 2,
        BOARD_SIZE * CELL_SIZE + PADDING * 2)
    )
    pygame.display.set_caption("Go 19x19 (Interactive)")
    font = pygame.font.SysFont(None, 28)


def neighbors(x, y):
    for nx, ny in [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]:
        if 0 <= nx < BOARD_SIZE and 0 <= ny < BOARD_SIZE:
            yield nx, ny


def get_group(board, x, y):
    """Возвращает группу камней и их свободы"""
    color = board[y][x]
    visited = set()
    group = set()
    liberties = set()

    stack = [(x, y)]
    while stack:
        cx, cy = stack.pop()
        if (cx, cy) in visited:
            continue
        visited.add((cx, cy))
        group.add((cx, cy))

        for nx, ny in neighbors(cx, cy):
            if board[ny][nx] == EMPTY:
                liberties.add((nx, ny))
            elif board[ny][nx] == color:
                stack.append((nx, ny))

    return group, liberties


def remove_dead_stones(board, color):
    """Удаляет группы указанного цвета без свобод"""
    removed = False
    to_remove = []

    for y in range(BOARD_SIZE):
        for x in range(BOARD_SIZE):
            if board[y][x] == color:
                group, libs = get_group(board, x, y)
                if len(libs) == 0:
                    to_remove.extend(group)
                    removed = True

    for x, y in to_remove:
        board[y][x] = EMPTY

    return removed


def is_suicide(board, x, y, color):
    """Проверка, что ход не самоубийственный"""
    temp_board = [row[:] for row in board]
    temp_board[y][x] = color

    # Сначала проверяем, у группы есть свободы
    _, libs = get_group(temp_board, x, y)
    if len(libs) > 0:
        return False

    # Если нет свобод — возможно, мы убиваем соперника → ход разрешён
    enemy = WHITE if color == BLACK else BLACK
    removed = False

    for nx, ny in neighbors(x, y):
        if temp_board[ny][nx] == enemy:
            g, libs = get_group(temp_board, nx, ny)
            if len(libs) == 0:
                removed = True

    return not removed


def try_place_stone(board, x, y, color):
    if board[y][x] != EMPTY:
        return False

    if is_suicide(board, x, y, color):
        return False

    # Ставим камень
    board[y][x] = color

    # Удаляем убитые группы противника
    enemy = WHITE if color == BLACK else BLACK
    remove_dead_stones(board, enemy)

    return True


def draw_board(board, turn):
    screen.fill((210, 170, 110))

    # Линии
    for i in range(BOARD_SIZE):
        pygame.draw.line(
            screen,
            (0, 0, 0),
            (PADDING, PADDING + i * CELL_SIZE),
            (PADDING + (BOARD_SIZE - 1) * CELL_SIZE, PADDING + i * CELL_SIZE),
            1
        )
        pygame.draw.line(
            screen,
            (0, 0, 0),
            (PADDING + i * CELL_SIZE, PADDING),
            (PADDING + i * CELL_SIZE, PADDING + (BOARD_SIZE - 1) * CELL_SIZE),
            1
        )

    # Хоcи (точки)
    hoshi = [3, 9, 15]
    for hx in hoshi:
        for hy in hoshi:
            pygame.draw.circle(
                screen, (0, 0, 0),
                (PADDING + hx * CELL_SIZE, PADDING + hy * CELL_SIZE),
                4
            )

    # Камни
    for y in range(BOARD_SIZE):
        for x in range(BOARD_SIZE):
            if board[y][x] == BLACK:
                pygame.draw.circle(
                    screen,
                    (0, 0, 0),
                    (PADDING + x * CELL_SIZE, PADDING + y * CELL_SIZE),
                    STONE_RADIUS
                )
            elif board[y][x] == WHITE:
                pygame.draw.circle(
                    screen,
                    (255, 255, 255),
                    (PADDING + x * CELL_SIZE, PADDING + y * CELL_SIZE),
                    STONE_RADIUS
                )

    turn_text = "Ход: Черные" if turn == BLACK else "Ход: Белые"
    t = font.render(turn_text, True, (0, 0, 0))
    screen.blit(t, (10, 10))

    pygame.display.flip()

def player_step(board, events):
    for event in events:
        if event.type ==  event.type == pygame.MOUSEBUTTONDOWN:
            mx, my = event.pos

            # Находим ближайшую точку пересечения
            x = round((mx - PADDING) / CELL_SIZE)
            y = round((my - PADDING) / CELL_SIZE)

            if 0 <= x < BOARD_SIZE and 0 <= y < BOARD_SIZE:
                return x, y
    return -1, -1

def ai_step():
    pass

def train():
    while True:
        x, y = None, None
        while True:
            x, y = ai_step(board, turn)
            if try_place_stone(board, x, y, turn):
                turn = WHITE if turn == BLACK else BLACK

def test():
    clock = pygame.time.Clock()

    while True:
        draw_board(board, turn)
        x, y = None, None
        events = pygame.event.get()
        for event in events:
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
        while True:
            if turn == BLACK and HAS_PLAYER:
                x, y = player_step(board, events)
                if x == y == -1:
                    break
            else:
                x, y = ai_step(board, turn)
                if try_place_stone(board, x, y, turn):
                    turn = WHITE if turn == BLACK else BLACK
            break
        clock.tick(30)


board = [[EMPTY for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]
turn = BLACK

def main():
    if TRAIN:
        train()
    else:
        test()


main()
